This notebook loads the preprocessed data created by `create_sdt.py` and `fingerprint_docs.ipynb` and then splits it into train, validate, and test sets.

In [1]:
import pickle


with open('docs.pkl', 'rb') as f:
    docs = pickle.load(f)
print('%i documents/data points' % len(docs))

with open('sdt.pkl', 'rb') as f:
    sdts = pickle.load(f)

with open('fingerprints.pkl', 'rb') as file_handle:
    fingerprints = pickle.load(file_handle)

17019 documents/data points


In [2]:
%load_ext ipycache

/global/homes/k/ktran/miniconda3/envs/gaspy/lib/python3.6/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated since IPython 4.0. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/global/homes/k/ktran/miniconda3/envs/gaspy/lib/python3.6/site-packages/ipycache.py:17: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  from IPython.utils.traitlets import Unicode


In [3]:
%%cache splits.pkl docs_train docs_val docs_test sdts_train sdts_val sdts_test fingerprints_train fingerprints_val fingerprints_test targets_train targets_val targets_test

import numpy as np
from sklearn.model_selection import train_test_split


targets = np.array([doc['energy'] for doc in docs]).reshape(-1, 1)

(docs_cv, docs_test,
 sdts_cv, sdts_test,
 fingerprints_cv, fingerprints_test,
 targets_cv, targets_test) = train_test_split(docs, sdts, fingerprints, targets, test_size=0.2)

(docs_train, docs_val,
 sdts_train, sdts_val,
 fingerprints_train, fingerprints_val,
 targets_train, targets_val) = train_test_split(docs_cv, sdts_cv, fingerprints_cv, targets_cv, test_size=0.2)

[Saved variables 'docs_test, docs_train, docs_val, fingerprints_test, fingerprints_train, fingerprints_val, sdts_test, sdts_train, sdts_val, targets_test, targets_train, targets_val' to file '/global/u2/k/ktran/sandbox/2019/08_Aug/27_redo_ml_benchmarking/preprocessing/splits.pkl'.]
